In [1]:
import pandas as pd
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score

In [2]:
df = pd.read_csv('mortgage.csv')

In [3]:
df.isna().sum()

id                           0
time                         0
orig_time                    0
first_time                   0
mat_time                     0
balance_time                 0
LTV_time                   270
interest_rate_time           0
hpi_time                     0
gdp_time                     0
uer_time                     0
REtype_CO_orig_time          0
REtype_PU_orig_time          0
REtype_SF_orig_time          0
investor_orig_time           0
balance_orig_time            0
FICO_orig_time               0
LTV_orig_time                0
Interest_Rate_orig_time      0
hpi_orig_time                0
default_time                 0
payoff_time                  0
status_time                  0
dtype: int64

In [4]:
#drop rows with null values, only 270/600K and represents proportionate amount of delinquent loans
df = df.dropna()

In [5]:
df['default'] = df['default_time']

In [6]:
del df['default_time']

In [10]:
#split data between trainin and testing sets to maintain blind testing
X_tr, X_te, y_tr, y_te = train_test_split(df.iloc[:, :22], df.iloc[:, 22], 
                                                    test_size = 0.2, random_state=42)

In [11]:
import imblearn.over_sampling

In [12]:
smote = imblearn.over_sampling.SMOTE(\
                                               ratio={0:485727,1:12048*40}, \
                                               random_state=42) 
X_tr_smote, y_tr_smote = smote.fit_sample(X_tr, y_tr)

lr_smote = LogisticRegression() 
lr_smote.fit(X_tr_smote, y_tr_smote)

print('Logistic Regression on SMOTE Train Data; Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(lr_smote.predict(X_te), y_te), roc_auc_score(y_te, lr_smote.predict_proba(X_te)[:,1])))

/Users/christianbuerkle/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression on SMOTE Train Data; Test F1: 0.657, Test AUC: 1.000


In [14]:
#confusion matrix on test data
confusion_matrix(y_te, lr_smote.predict(X_te))

array([[118103,   3236],
       [     4,   3101]])

In [16]:
# Accuracy of my classification model
lr_smote.score(X_test, y_test)

0.9739641927292597

In [33]:
lr_40x = LogisticRegression(class_weight={1 : 40, 0 : 1})

In [34]:
lr_40x.fit(X_tr, y_tr) 
print('40:1 class weights Logistic Regression Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(lr_4x.predict(X_te), y_te), roc_auc_score(y_te, lr_4x.predict_proba(X_te)[:,1])))

/Users/christianbuerkle/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


40:1 class weights Logistic Regression Test F1: 0.999, Test AUC: 1.000


/Users/christianbuerkle/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [35]:
confusion_matrix(y_te, lr_40x.predict(X_te))

array([[121333,      6],
       [     0,   3105]])

In [38]:
y_te.shape

(124444,)

In [39]:
y_te.value_counts()

0    121339
1      3105
Name: default, dtype: int64